In [1]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
%matplotlib inline 

pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 50)

# import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from tqdm import tqdm_notebook

from itertools import product


def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [2]:
sales = pd.read_csv('sales_train.csv.gz')
shops = pd.read_csv('shops.csv')
items = pd.read_csv('items.csv')
item_cats = pd.read_csv('item_categories.csv')

In [3]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
test_daily = pd.read_csv('test_daily.csv')
test_daily['item_price'] = pd.Series(np.nan, index=test_daily.index)
test_daily['item_cnt_day'] = pd.Series(np.nan, index=test_daily.index)
test_daily['date'] = test_daily['date'].apply(lambda d: d.replace('-', '.'))
# test_daily.head()
sales = pd.concat([sales, test_daily])
sales.tail()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
6425995,30.11.2015,34,45,18454,NaN,NaN
6425996,30.11.2015,34,45,16188,NaN,NaN
6425997,30.11.2015,34,45,15757,NaN,NaN
6425998,30.11.2015,34,45,19648,NaN,NaN
6425999,30.11.2015,34,45,969,NaN,NaN


In [5]:
# Prepare features
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# Groupby data to get shop-item-month aggregates
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
# Fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values] 
# Join it to the grid
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

# Same as above but with shop-month aggregates
gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_shop':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

# Same as above but with item-month aggregates
gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_item':'sum'}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

# Downcast dtypes from 64 to 32 bit to save memory
all_data = downcast_dtypes(all_data)
del grid, gb 
gc.collect();

/home/baptiste/miniconda3/envs/coursera/lib/python3.6/site-packages/pandas/core/groupby.py:4189: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [6]:
# List of columns that we will use to create lags
cols_to_rename = list(all_data.columns.difference(index_cols)) 

shift_range = [1, 2, 6, 12]

for month_shift in tqdm_notebook(shift_range):
    train_shift = all_data[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)

del train_shift

# Don't use old data from year 2013
all_data = all_data[all_data['date_block_num'] >= 12] 

# List of all lagged features
fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]] 
# We will drop these at fitting stage
to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 

# Category for each item
item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')
all_data = downcast_dtypes(all_data)
gc.collect();

A Jupyter Widget

In [7]:
# Save `date_block_num`, as we can't use them as features, but will need them to split the dataset into parts 
dates = all_data['date_block_num']

last_block = 33 #dates.max()
print('Test `date_block_num` is %d' % last_block)

Test `date_block_num` is 33


In [8]:
dates_train = dates[dates <  last_block]
dates_test  = dates[dates == last_block]

X_train = all_data.loc[dates <  last_block].drop(to_drop_cols, axis=1)
X_test =  all_data.loc[dates == last_block].drop(to_drop_cols, axis=1)

y_train = all_data.loc[dates <  last_block, 'target']
y_test =  all_data.loc[dates == last_block, 'target']

X_preds = all_data.loc[dates == 34].drop(to_drop_cols, axis=1)

In [9]:
lr = LinearRegression()
lr.fit(X_train.values, y_train)
pred_lr = lr.predict(X_test.values)

print('Test R-squared for linreg is %f' % r2_score(y_test, pred_lr))
# Test R-squared for linreg is 0.219363 for 1,2,6,12 month lag

Test R-squared for linreg is 0.225637


In [10]:
reg_model = xgb.XGBRegressor(learning_rate=0.01, max_depth=10, min_child_weight=1, n_estimators=300, n_jobs=-1, shuffle=False)
reg_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       shuffle=False, silent=True, subsample=1)

In [11]:
preds = reg_model.predict(X_test)
print('Test R-squared for XGBoost is %f' % r2_score(y_test, preds))
print('Test RMSE for XGBoost is %f' % np.sqrt(mean_squared_error(y_test, preds)))
# Test R-squared for XGBoost is 0.210408
# Test RMSE for XGBoost is 4.747507
# with learning_rate = 0.01 and n_estimators =300, lags = 1,2,6,12
# Test R-squared for XGBoost is 0.209921
# Test RMSE for XGBoost is 4.748967

Test R-squared for XGBoost is 0.209921
Test RMSE for XGBoost is 4.748967


In [12]:
preds_kaggle = reg_model.predict(X_preds)
X_preds['preds'] = preds_kaggle
X_preds.head()

,shop_id,item_id,target_lag_1,target_item_lag_1,target_shop_lag_1,target_lag_2,target_item_lag_2,target_shop_lag_2,target_lag_3,target_item_lag_3,target_shop_lag_3,target_lag_6,target_item_lag_6,target_shop_lag_6,target_lag_12,target_item_lag_12,target_shop_lag_12,item_category_id,preds
6425094,5,5037,0.0,25.0,1052.0,1.0,110.0,1092.0,3.0,119.0,1294.0,1.0,87.0,1012.0,1.0,65.0,1445.0,19,0.406204
6425095,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55,0.214632
6425096,5,5233,1.0,42.0,1052.0,3.0,80.0,1092.0,1.0,150.0,1294.0,3.0,71.0,1012.0,0.0,0.0,0.0,19,0.908723
6425097,5,5232,0.0,28.0,1052.0,0.0,48.0,1092.0,1.0,65.0,1294.0,0.0,0.0,0.0,0.0,0.0,0.0,23,0.353434
6425098,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,4.033569


In [13]:
preds_gb = X_preds[['shop_id', 'item_id', 'preds']].groupby(['shop_id', 'item_id'], as_index=False).sum()
preds_gb['preds'] = preds_gb['preds'].clip(0, 20)
preds_gb.rename(columns={'preds': 'item_cnt_month'}, inplace=True)
preds_gb.head()

,shop_id,item_id,item_cnt_month
0,2,30,0.041793
1,2,31,0.216916
2,2,32,0.194747
3,2,33,0.213736
4,2,38,0.042198


In [14]:
(pd.read_csv('test.csv')
 .merge(preds_gb, on=['shop_id', 'item_id'], how='left')
 .drop(['shop_id', 'item_id'], axis=1)
 .to_csv('xgb_preds_300est.csv', index=False)
)

In [15]:
%ls

all_data_wk4.csv        previous_values.csv       test.csv.gz
all_ones.csv            previous_xgb.csv          test_daily.csv
EDA_1st_week.ipynb      sales_train.csv.gz        test_to_daily.ipynb
ideas.txt               sales_train_v2.csv        train_test.csv
item_categories.csv     sample_submission.csv     xgb_preds_300est.csv
items.csv               sample_submission.csv.gz  XGBreg_daily.ipynb
new_xgb_preds.csv       shops.csv                 XGBreg_monthly.ipynb
predict_wk4-best.ipynb  submission.csv            X_train_wk4.csv
predict_wk4.ipynb       test.csv


In [16]:
# add lagged mean item price, min, max of item price, gridsearch for n_estimators and learning rate
# ensembling